In [1]:
#imports
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib as plt
import pandas as pd
import geopandas as gpd
import time
%matplotlib inline

In [7]:
#import credentials from cred.py file
from creds import creds

In [8]:
#establish database connection
conn = psycopg2.connect( host=creds['hostname'], user=creds['username'], password=creds['password'], dbname=creds['database'] )
#conn.close()

### Intersection Query

Find the intersection between geolife_data and osm_pois_buffer_ref

In [10]:
start = time.time()
intersection_query = """SELECT * FROM public.osm_pois_ref limit 10"""
intersection_frame = pd.read_sql_query(intersection_query ,conn)
elapsed_time_fl = (time.time() - start) 
print('Time for Intersection: {} s'.format(elapsed_time_fl))

0.12932634353637695


### Spatial Join

In [ ]:
start = time.time()
join_query = """SELECT * FROM public.osm_pois_ref limit 10"""
join_frame = pd.read_sql_query(join_query ,conn)
elapsed_time_fl = (time.time() - start) 
print('Time for Spatial Join: {} s'.format(elapsed_time_fl))

### Spatial Clustering

In [ ]:
start = time.time()
spatial_query = """SELECT *, ST_ClusterDBSCAN(geom, eps := 300, minPoints := 1000)
                OVER(ORDER BY user_id) AS cluster_id FROM geolife_data
                LIMIT 8;"""
spatial_clusters = pd.read_sql_query(spatial_query ,conn)
elapsed_time_fl = (time.time() - start) 
print('Time for Spatial Clustering: {} s'.format(elapsed_time_fl))

In [11]:
#set up a geopandas dataframe from sql query
#osm_pois_gpd = gpd.read_postgis(sql = osm_query, con = conn)
#osm_pois_gpd.head()

In [ ]:
conn.close()